In [ ]:
# https://huggingface.co/spaces/Norod78/sd2-simpsons-blip/blob/main/app.py

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
%matplotlib widget
from matplotlib import pyplot as plt

from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
import torch
from PIL import Image

model_id = 'Norod78/sd2-simpsons-blip'
prefix = None
     
scheduler = DPMSolverMultistepScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    num_train_timesteps=1000,
    trained_betas=None,
    #predict_epsilon=True,
    prediction_type='epsilon',
    thresholding=False,
    algorithm_type="dpmsolver++",
    solver_type="midpoint",
    lower_order_final=True,
)

use_cuda = False and torch.cuda.is_available()
pipe = StableDiffusionPipeline.from_pretrained(
  model_id,
  torch_dtype=torch.float16 if use_cuda else torch.float32,
  scheduler=scheduler)
pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
  model_id,
  torch_dtype=torch.float16 if use_cuda else torch.float32,
  scheduler=scheduler)
if use_cuda:
  pipe = pipe.to("cuda")
  pipe.enable_attention_slicing()
  pipe_i2i = pipe_i2i.to("cuda")


def inference(prompt, guidance, steps, width=512, height=512, seed=0, img=None, strength=0.5, neg_prompt=""):
  generator = None
  if use_cuda:
    generator = torch.Generator('cuda').manual_seed(seed) if seed else None
  elif seed:      
    generator = torch.Generator()
    generator.manual_seed(seed)
  try:
    if img is not None:
      #return img_to_img(prompt, neg_prompt, img, strength, guidance, steps, width, height, generator), None
      return img_to_img(prompt, neg_prompt, img, strength, guidance, steps, width, height, generator)
    #return txt_to_img(prompt, neg_prompt, guidance, steps, width, height, generator), None
    return txt_to_img(prompt, neg_prompt, guidance, steps, width, height, generator)
  except Exception as e:
    raise
    #return None, str(e)

def txt_to_img(prompt, neg_prompt, guidance, steps, width, height, generator):
    result = pipe(
      prompt,
      negative_prompt = neg_prompt,
      num_inference_steps = int(steps),
      guidance_scale = guidance,
      width = width,
      height = height,
      generator = generator)
    #return replace_nsfw_images(result)
    return result.images[0]

def img_to_img(prompt, neg_prompt, img, strength, guidance, steps, width, height, generator):
    ratio = min(height / img.height, width / img.width)
    img = img.resize((int(img.width * ratio), int(img.height * ratio)), Image.LANCZOS)
    result = pipe_i2i(
        prompt,
        negative_prompt = neg_prompt,
        init_image = img,
        num_inference_steps = int(steps),
        strength = strength,
        guidance_scale = guidance,
        width = width,
        height = height,
        generator = generator)
    #return replace_nsfw_images(result)
    return result.images[0]

#def replace_nsfw_images(results):
#    for i in range(len(results.images)):
#      if 'nsfw_content_detected' in results and results.nsfw_content_detected[i]:
#        results.images[i] = Image.open("nsfw.png")
#    return results.images[0]

def UI():
  import gradio as gr
  css = """.main-div div{display:inline-flex;align-items:center;gap:.8rem;font-size:1.75rem}.main-div div h1{font-weight:900;margin-bottom:7px}.main-div p{margin-bottom:10px;font-size:94%}a{text-decoration:underline}.tabs{margin-top:0;margin-bottom:0}#gallery{min-height:20rem}"""
  with gr.Blocks(css=css, analytics_enabled=False) as demo:
    gr.HTML(f"""
        <div class="main-div">
          <div>
            <h1>SDv2 Simpsons</h1>
          </div>
          <p>
           Demo for <a href="https://huggingface.co/Norod78/sd2-simpsons-blip">SD2 Simpsons BLIP</a> Stable Diffusion 2, fine-tuned model.<br>
           {"Add the following tokens to your prompts for the model to work properly: <b>prefix</b>" if prefix else ""}
          </p>
          Running on {"<b>GPU 🔥</b>" if use_cuda else f"<b>CPU 🥶</b>. For faster inference it is recommended to <b>upgrade to GPU in <a href='https://huggingface.co/spaces/Norod78/sd2-simpsons-blip/settings'>Settings</a></b>"}<br><br>
          <a style="display:inline-block" href="https://huggingface.co/spaces/Norod78/sd2-simpsons-blip?duplicate=true"><img src="https://bit.ly/3gLdBN6" alt="Duplicate Space"></a>
        </div>""")
    with gr.Row():
        with gr.Column(scale=55):
          with gr.Group():
              with gr.Row():
                prompt = gr.Textbox(label="Prompt", show_label=False, max_lines=2,placeholder="[your prompt]").style(container=False)
                generate = gr.Button(value="Generate").style(rounded=(False, True, True, False))
              image_out = gr.Image(height=512)
          error_output = gr.Markdown()
        with gr.Column(scale=45):
          with gr.Tab("Options"):
            with gr.Group():
              neg_prompt = gr.Textbox(label="Negative prompt", placeholder="What to exclude from the image")              
              with gr.Row():
                guidance = gr.Slider(label="Guidance scale", value=7.5, maximum=15)
                steps = gr.Slider(label="Steps", value=25, minimum=2, maximum=75, step=1)
              with gr.Row():
                width = gr.Slider(label="Width", value=512, minimum=64, maximum=1024, step=8)
                height = gr.Slider(label="Height", value=512, minimum=64, maximum=1024, step=8)
              seed = gr.Slider(0, 2147483647, label='Seed (0 = random)', value=0, step=1)
          with gr.Tab("Image to image"):
              with gr.Group():
                image = gr.Image(label="Image", height=256, tool="editor", type="pil")
                strength = gr.Slider(label="Transformation strength", minimum=0, maximum=1, step=0.01, value=0.5)

    inputs = [prompt, guidance, steps, width, height, seed, image, strength, neg_prompt]
    outputs = [image_out, error_output]
    prompt.submit(inference, inputs=inputs, outputs=outputs)
    generate.click(inference, inputs=inputs, outputs=outputs)
    #gr.HTML("""
    #    <div style="border-top: 1px solid #303030;">
    #      <br>
    #      <p>This space was created using <a href="https://huggingface.co/spaces/anzorq/sd-space-creator">SD Space Creator</a>.</p>
    #    </div>""")
    demo.queue(concurrency_count=1)
    demo.launch(debug=True, share=False, height=768, show_api=False)

def manual():
    prompt = "mouse with a blue sign"
    guidance = 7.5 # max 15
    steps = 25 # [2, 75]
    width = 768
    height = 768
    seed = 0
    image = None
    strength = 0.5 # [0, 1]
    neg_prompt = ""
    image_out = inference(prompt, guidance, steps, width, height, seed, image, strength, neg_prompt)
    plt.figure()
    plt.imshow(image_out)
    plt.axis("off")


#UI()
manual()